In [1]:
"""Load CAS Hamiltonians
"""
import saveload_utils as sl
import ferm_utils as feru
import csa_utils as csau
import var_utils as varu
import openfermion as of
import numpy as np
from sdstate import *
from itertools import product
import random
import h5py
import sys
from matrix_utils import construct_orthogonal
import pickle
import tensorflow as tf

def get_cas_matrix(cas_x, n, k):
    obt = np.zeros([n, n])
    tbt = np.zeros([n, n, n, n])
    idx = 0
    for orbs in k:
        for p, q in product(orbs, repeat = 2):
            obt[p,q] = cas_x[idx]
            idx += 1
        for p, q, r, s in product(orbs, repeat = 4):
            tbt[p,q,r,s] = cas_x[idx]
            idx += 1
    return obt, tbt

def orbtransf(tensor, U, complex = False):
    """Return applying UHU* for the tensor representing the 1e or 2e tensor"""
    if len(tensor.shape) == 4:
        p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor)[0]
        return np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor, optimize = p)
    elif len(tensor.shape) == 2:
        p = np.einsum_path('ap,bq, pq->ab', U, U, tensor)[0]
        return np.einsum('ap,bq, pq->ab', U, U, tensor, optimize = p)
def load_Hamiltonian_with_solution(path, file_name):
    """Load the precomputed Hamiltonian with the given file_name. Return the Hamiltonians and the state
    Hamiltonian is represented in spatial orbital basis as three terms: (c, h_pq, g_pqrs)
    Returns:    
    U: Arbitrary Unitary Rotation, in spatial orbital basis
    H_cas: Unhidden CAS Fragements
    H_hidden：U H_cas U*
    H_with_killer: H_cas + killer
    H_killer_hidden: U H_with_killer U*
    """
    with open(ps_path + f_name, 'rb') as handle:
        dic = pickle.load(handle)
        key = list(dic.keys())[0]
        dic = dic[key]
    cas_x = dic["cas_x"]
    killer = dic["killer"]
    killer_c = killer[0]
    sol = dic["sol"]
    k_obt = tf.sparse.reorder(killer[1])
    k_tbt = tf.sparse.reorder(killer[2])
    k = dic["k"]
    upnum = dic["upnum"]
    spatial_orbs = dic["spatial_orbs"]
#     CAS 2e tensor
    obt, tbt = get_cas_matrix(cas_x, spatial_orbs, k)
    H_cas = (0, obt, tbt)
    H_with_killer = (killer_c, obt+tf.sparse.to_dense(k_obt), tbt+tf.sparse.to_dense(k_tbt))
#     Set up random unitary to hide 2e tensor
    random_uparams = np.random.rand(upnum)
    U = construct_orthogonal(spatial_orbs, random_uparams)
#     Hide 2e etensor with random unitary transformation
    H_hidden = (0, orbtransf(obt, U), orbtransf(tbt, U))
    H_killer_hidden = (killer_c, orbtransf(H_with_killer[1], U), orbtransf(H_with_killer[2], U))
    return U, H_cas, H_hidden, H_with_killer, H_killer_hidden, sol

ps_path = "planted_solutions/"
file_name = "2_co2_6-311++G___12_9d464efb-b312-45f8-b0ba-8c42663059dc.hdf5"
f_name = file_name.split(".")[0] + ".pkl"
print(f_name)
U, H_cas, H_hidden, H_with_killer, H_killer_hidden, sol = load_Hamiltonian_with_solution(ps_path, f_name)

2_co2_6-311++G___12_9d464efb-b312-45f8-b0ba-8c42663059dc.pkl


In [5]:
print(U.shape)
print(H_cas[1].shape)

(12, 12)
(12, 12)
